In [1]:
import hopsworks
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/Users/karlsimu/miniforge3/envs/base-id2223/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### put full model in dir

In [ ]:
import torch
from CNN_model import ConvNet
# save entire model, not just state dict
device = torch.device('cpu')
model = ConvNet(num_class=1000)
m = torch.load('best_model/model', map_location=device)
model.load_state_dict(m['model'])

entire_model_path = 'best_model/entire_model'
torch.save(model, entire_model_path)

## HOPsworks

In [2]:
project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/185498


In [3]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


## upload model

In [4]:
model_dir="best_model"

In [5]:
sound_model = mr.python.create_model(
    name="freesound_model", 
    description="Audio tagging"
)

In [6]:
sound_model.save(model_dir)

Model export complete: 100%|██████████| 6/6 [09:17<00:00, 92.90s/it]                    

Model created, explore it at https://c.app.hopsworks.ai:443/p/185498/models/freesound_model/4


Model(name: 'freesound_model', version: 4)

## test fetch model

In [9]:
import torch
model_registry = project.get_model_registry()
model_p = model_registry.get_model("freesound_model", version=4)
model_dir = model_p.download()

model_path = model_dir + "/entire_model"
scaler_path = model_dir + "/scaler_top_1000.pkl"
id2labels_path = model_dir + "/id2token_top_1000.json"
py_file_path = model_dir + "/CNN_model.py"

os.replace(py_file_path, "CNN_model.py")

model = torch.load(model_path)
model.eval()

Connected. Call `.close()` to terminate connection gracefully.

ConvNet(
  (bn_init): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mp_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

## Test upload sound

In [7]:
resource_path = "Resources/Freesound/"
s_file_name = resource_path + "sound_1.ogg"
dataset_api = project.get_dataset_api()
dataset_api.upload(s_file_name, resource_path, overwrite=True)

Uploading: 100.000%|██████████| 22432/22432 elapsed<00:01 remaining<00:00


'Resources/Freesound//sound_1.ogg'

In [8]:
resource_path = "Resources/Freesound/"
s_file_name = resource_path + "sound_1.ogg"
s_file_path = dataset_api.download(s_file_name, resource_path, overwrite=True)


Downloading: 100.000%|██████████| 22432/22432 elapsed<00:00 remaining<00:00


## Upload json

In [3]:

import json 

dataset_api = project.get_dataset_api()

# create
prd_lbl_tokens_dict = {717730 : ['impulse', 'sine', 'multisample', 'pop', 'electro']} 
prd_lbl_tokens_json = json.dumps(prd_lbl_tokens_dict)

local_path = "Resources/Freesound/prd_lbl_tokens.json"
remote_path = "Resources/Freesound/"

# Write JSON object to file
with open("Resources/Freesound/prd_lbl_tokens.json", "w") as of: 
    of.write(prd_lbl_tokens_json)

dataset_api.upload(local_path, remote_path, overwrite=True)

Uploading: 100.000%|██████████| 64/64 elapsed<00:01 remaining<00:00


'Resources/Freesound//prd_lbl_tokens.json'

In [4]:
#these are switched
remote_path = "Resources/Freesound/prd_lbl_tokens.json"
local_path = "Resources/Freesound/"

#Download
prd_lbl_tokens_file_path = dataset_api.download(remote_path, local_path, overwrite=True)

Downloading: 100.000%|██████████| 64/64 elapsed<00:00 remaining<00:00


In [5]:
# add new entry
local_path = "Resources/Freesound/prd_lbl_tokens.json"
with open(local_path) as jsf:
    prd_lbl_tokens_dict = json.load(jsf)
prd_lbl_tokens_dict['717731'] = ['impulse', 'sine', 'multisample', 'pop', 'electro']
prd_lbl_tokens_dict

{'717730': ['impulse', 'sine', 'multisample', 'pop', 'electro'],
 '717731': ['impulse', 'sine', 'multisample', 'pop', 'electro']}

In [6]:
local_path = "Resources/Freesound/prd_lbl_tokens.json"
remote_path = "Resources/Freesound/"

# overwrite old local
prd_lbl_tokens_json = json.dumps(prd_lbl_tokens_dict)
with open(local_path, "w") as of: 
    of.write(prd_lbl_tokens_json)

# upload
dataset_api.upload(local_path, remote_path, overwrite=True)

Uploading: 100.000%|██████████| 128/128 elapsed<00:01 remaining<00:00


'Resources/Freesound//prd_lbl_tokens.json'

In [3]:

import os
remote_path = "dira/dirb/"
if not os.path.exists(remote_path):
    os.makedirs(remote_path)
    print('made dir')

made dir
